In [1]:
import pandas as pd
from PIL import Image
from skimage import color

import numpy as np
from colour import XYZ_to_CAM16
import numpy as np
from colour.models import RGB_to_XYZ
# 读取图片
img = Image.open('/home/longmenzhen/项目/wind/ss.png')

# 获取图片尺寸
width, height = img.size

# 获取像素数据
pixels = list(img.getdata())

# 创建DataFrame,A代表透明度
try:
    df = pd.DataFrame(pixels, columns=['R', 'G', 'B','A'])
    df = df.drop(columns=['A'])
except:
    df = pd.DataFrame(pixels, columns=['R', 'G', 'B'])
# 添加X和Y列
df['X'] = df.index % width
df['Y'] = df.index // width

# 显示DataFrame
print(df)

          R    G    B    X    Y
0        34   96  147    0    0
1        39  101  148    1    0
2        28   92  130    2    0
3        37  102  130    3    0
4        20   87  104    4    0
...     ...  ...  ...  ...  ...
395995  108   67   45  490  799
395996  134   79   49  491  799
395997  160   84   52  492  799
395998  167   72   44  493  799
395999  172   60   46  494  799

[396000 rows x 5 columns]


In [2]:
from colorspacious import cspace_converter
RGB = np.array([df['R'], df['G'], df['B']]).T

converter = cspace_converter("sRGB255", "CIELCh")
# 将LCH颜色添加到DataFrame
lch = converter(RGB)
df['L'] = lch[:, 0]
df['C'] = lch[:, 1]
df['H'] = lch[:, 2]
df['L'] = df['L'].round()
df['C'] = df['C'].round()
df['H'] = df['H'].round()
df['count'] = 1
df


,R,G,B,X,Y,L,C,H,count
0,34,96,147,0,0,39.0,34.0,268.0,1
1,39,101,148,1,0,41.0,32.0,264.0,1
2,28,92,130,2,0,37.0,28.0,257.0,1
3,37,102,130,3,0,40.0,24.0,243.0,1
4,20,87,104,4,0,34.0,21.0,228.0,1
...,...,...,...,...,...,...,...,...,...
395995,108,67,45,490,799,33.0,26.0,54.0,1
395996,134,79,49,491,799,39.0,34.0,54.0,1
395997,160,84,52,492,799,44.0,43.0,49.0,1
395998,167,72,44,493,799,43.0,51.0,43.0,1


In [3]:
C_mean = df['C'].mean()
L_mean = df['L'].mean()
H_mean = df['H'].mean()

In [4]:
df_mean = df.groupby('H').agg({'R': 'mean', 'G': 'mean', 'B': 'mean','L': 'mean','C':'mean'}).reset_index()
df_mean['R'] = df_mean['R'].round().astype(int)
df_mean['G'] = df_mean['G'].round().astype(int)
df_mean['B'] = df_mean['B'].round().astype(int)
df_mean['L'] = df_mean['L'].round().astype(int)
df_mean['C'] = df_mean['C'].round().astype(int)
df_count = df.groupby([ 'H']).size().reset_index(name='v')
df_merged = pd.merge(df_mean, df_count, on='H')
df_merged

,H,R,G,B,L,C,v
0,0.0,100,64,76,31,16,1511
1,1.0,111,63,79,33,22,2032
2,2.0,124,64,85,36,27,1376
3,3.0,144,76,98,42,30,1359
4,4.0,143,87,104,44,24,2340
...,...,...,...,...,...,...,...
356,356.0,91,58,71,28,16,2481
357,357.0,78,42,56,22,17,2206
358,358.0,65,30,43,16,17,2419
359,359.0,104,60,76,30,20,1713


In [5]:
import plotly.graph_objects as go

# Sort the DataFrame by 'H' column
df_merged_sorted = df_merged.sort_values('H')

colors = ['rgb({},{},{})'.format(r, g, b) for r, g, b in zip(df_merged_sorted['R'], df_merged_sorted['G'], df_merged_sorted['B'])]


fig = go.Figure(data=go.Pie(labels=df_merged_sorted['H'], values=df_merged_sorted['v'], marker=dict(colors=colors), textinfo='none', direction='clockwise', sort=False))
fig.update_layout(title='色相分布图（按色相顺序）', showlegend=False)
fig.show()

In [6]:
fig = go.Figure(data=go.Pie(labels=df_merged_sorted['H'], values=df_merged_sorted['v'], marker=dict(colors=colors), textinfo='none'))
fig.update_layout(title='色相分布图（按每个色相份额排序）', showlegend=False)
fig.show()

In [7]:
import plotly.graph_objects as go

# Sort the DataFrame by 'H' column
df_merged_sorted = df_merged.sort_values('L')

colors = ['rgb({},{},{})'.format(r, g, b) for r, g, b in zip(df_merged_sorted['R'], df_merged_sorted['G'], df_merged_sorted['B'])]


fig = go.Figure(data=go.Pie(labels=df_merged_sorted['H'], values=df_merged_sorted['v'], marker=dict(colors=colors), textinfo='none', direction='clockwise', sort=False))
fig.update_layout(title='色相分布图（按亮度排序）', showlegend=False)
fig.show()

In [8]:




import plotly.graph_objects as go

# Sort the DataFrame by 'H' column
df_merged_sorted = df_merged.sort_values('H')

# Calculate LCH values based on C_mean, L_mean, and H
lch_colors = [(L_mean, C_mean, h) for h in df_merged_sorted['H']]

# Convert LCH colors to RGB
converter = cspace_converter("CIELCh", "sRGB255")
rgb_colors = converter(lch_colors)

# Format RGB colors
colors = ['rgb({},{},{})'.format(int(r), int(g), int(b)) for r, g, b in rgb_colors]

fig = go.Figure(data=go.Pie(labels=df_merged_sorted['H'], values=df_merged_sorted['v'], marker=dict(colors=colors), textinfo='none', direction='clockwise', sort=False))
fig.update_layout(title='色相分布图(平均色度和亮度后)', showlegend=False)
fig.show()

In [9]:
df_learn = df.groupby(['C', 'H']).agg({ 'L': 'sum'}).reset_index()
df_learn.columns = ['C', 'H', 'L']
df_learn['C_36']= df_learn['C']*3.6

In [10]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=256, init='k-means++',algorithm='lloyd')
kmeans.fit(df_learn[['C_36', 'H',]], sample_weight=df_learn['L'])
# 获取聚类结果
df_predict = df.copy()
df_predict['C_36'] = df_predict['C']*3.6
labels = kmeans.predict(df_predict[['C_36', 'H']])
# 将聚类结果写回原始数据框中

/home/longmenzhen/.conda/envs/color/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/longmenzhen/.conda/envs/color/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/longmenzhen/.conda/envs/color/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/longmenzhen/.conda/envs/color/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/home/longmenzhen/.conda/envs/color/lib/python3.11/site-packages

In [11]:
df['Cluster'] = labels
df

,R,G,B,X,Y,L,C,H,count,Cluster
0,34,96,147,0,0,39.0,34.0,268.0,1,72
1,39,101,148,1,0,41.0,32.0,264.0,1,24
2,28,92,130,2,0,37.0,28.0,257.0,1,85
3,37,102,130,3,0,40.0,24.0,243.0,1,253
4,20,87,104,4,0,34.0,21.0,228.0,1,41
...,...,...,...,...,...,...,...,...,...,...
395995,108,67,45,490,799,33.0,26.0,54.0,1,242
395996,134,79,49,491,799,39.0,34.0,54.0,1,222
395997,160,84,52,492,799,44.0,43.0,49.0,1,101
395998,167,72,44,493,799,43.0,51.0,43.0,1,48


In [12]:
df_learn_show = df.groupby(['Cluster']).agg({ 'L': 'sum','C': 'mean','H': 'mean','R': 'mean', 'G': 'mean', 'B': 'mean','count': 'sum'}).reset_index()
df_learn_show['C'] = df_learn_show['C'].round().astype(int)
df_learn_show['H'] = df_learn_show['H'].round().astype(int)
df_learn_show['R'] = df_learn_show['R'].round().astype(int)
df_learn_show['G'] = df_learn_show['G'].round().astype(int)
df_learn_show['B'] = df_learn_show['B'].round().astype(int)

df_learn_show

,Cluster,L,C,H,R,G,B,count
0,0,110492.0,66,45,210,87,47,2098
1,1,110094.0,10,220,89,115,122,2365
2,2,127831.0,26,105,168,166,119,1905
3,3,35022.0,24,327,124,90,122,827
4,4,112451.0,24,13,141,86,93,2639
...,...,...,...,...,...,...,...,...
251,251,80677.0,8,281,83,86,99,2252
252,252,136551.0,30,125,145,167,115,2080
253,253,17244.0,22,246,75,126,154,343
254,254,14235.0,46,355,153,54,98,370


In [13]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=df_learn_show['C'],
    y=df_learn_show['H'],
    mode='markers',
    marker=dict(
        size=np.sqrt(df_learn_show['count'])/8,
        color=df_learn_show.apply(lambda row: f'rgb({row["R"]},{row["G"]},{row["B"]})', axis=1),
        opacity=0.8
    ),
    text=df_learn_show.apply(lambda row: f'R: {row["R"]}, G: {row["G"]}, B: {row["B"]}', axis=1)
))

fig.update_layout(
    title='对色彩进行聚类后的结果',
    xaxis=dict(title='C'),
    yaxis=dict(title='H'),
    showlegend=False
)

fig.show()


In [14]:
from sklearn.cluster import KMeans
kmeans2 = KMeans(n_clusters=5, init='k-means++',algorithm='lloyd')
kmeans2.fit(df_learn_show[['H','C']],sample_weight=df_learn_show['L'])
# 获取聚类结果
labels2 = kmeans2.predict(df_learn_show[[ 'H','C']])
df_learn_show['Cluster2'] = labels2
df_learn_show
# 将聚类结果写回原始数据框中 取出df_learn_show中cluster2相同的行中L值最大的行，用plotly绘制饼图，颜色用R，G，B值，大小用count值

/home/longmenzhen/.conda/envs/color/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/longmenzhen/.conda/envs/color/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/longmenzhen/.conda/envs/color/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/longmenzhen/.conda/envs/color/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/home/longmenzhen/.conda/envs/color/lib/python3.11/site-packages

,Cluster,L,C,H,R,G,B,count,Cluster2
0,0,110492.0,66,45,210,87,47,2098,1
1,1,110094.0,10,220,89,115,122,2365,0
2,2,127831.0,26,105,168,166,119,1905,4
3,3,35022.0,24,327,124,90,122,827,2
4,4,112451.0,24,13,141,86,93,2639,1
...,...,...,...,...,...,...,...,...,...
251,251,80677.0,8,281,83,86,99,2252,3
252,252,136551.0,30,125,145,167,115,2080,4
253,253,17244.0,22,246,75,126,154,343,3
254,254,14235.0,46,355,153,54,98,370,2


In [15]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=df_learn_show['C'],
    y=df_learn_show['H'],
    mode='markers',
    marker=dict(
        size=np.sqrt(df_learn_show['count'])/8,
        color=df_learn_show['Cluster2'],
        opacity=0.8
    ),
    text=df_learn_show.apply(lambda row: f'R: {row["R"]}, G: {row["G"]}, B: {row["B"]}', axis=1)
))

fig.update_layout(
    title='对色相聚一次类',
    xaxis=dict(title='C：色度'),
    yaxis=dict(title='H：色相'),
    showlegend=False
)

fig.show()

In [16]:
df_learn_show['final_v']=df_learn_show['C']*df_learn_show['L']

In [17]:
df_learn_l = df.groupby(['L']).agg({ 'count': 'sum'}).reset_index()

In [18]:
df_learn_show['L'] = (df_learn_show['L']/df_learn_show['count']).round().astype(int)

In [19]:
from sklearn.cluster import KMeans
kmeans3 = KMeans(n_clusters=10, init='k-means++',algorithm='lloyd')
kmeans3.fit(df_learn_l[['L']],sample_weight=df_learn_l['count'])
# 获取聚类结果
labels3 = kmeans3.predict(df_learn_l[[ 'L']])
df_learn_l['Cluster3'] = labels3
df_learn_l
# 将聚类结果写回原始数据框中 取出df_learn_show中cluster2相同的行中L值最大的行，用plotly绘制饼图，颜色用R，G，B值，大小用count值

/home/longmenzhen/.conda/envs/color/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/longmenzhen/.conda/envs/color/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/longmenzhen/.conda/envs/color/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

/home/longmenzhen/.conda/envs/color/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/home/longmenzhen/.conda/envs/color/lib/python3.11/site-packages

,L,count,Cluster3
0,0.0,379,1
1,1.0,1053,1
2,2.0,1631,1
3,3.0,2992,1
4,4.0,9180,1
...,...,...,...
96,96.0,35,2
97,97.0,17,2
98,98.0,21,2
99,99.0,16,2


In [20]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=df_learn_l['L'],
    y=df_learn_l['count'],
    mode='markers',
    marker=dict(
        color=df_learn_l['Cluster3'],
        colorscale='Viridis',
        opacity=0.8
    )
))

fig.update_layout(
    title='在亮度上对像素聚类',
    xaxis=dict(title='L：亮度'),
    yaxis=dict(title='像素量'),
    showlegend=False
)

fig.show()


In [21]:
df_learn_l

,L,count,Cluster3
0,0.0,379,1
1,1.0,1053,1
2,2.0,1631,1
3,3.0,2992,1
4,4.0,9180,1
...,...,...,...
96,96.0,35,2
97,97.0,17,2
98,98.0,21,2
99,99.0,16,2


In [22]:
df_l_plan = df_learn_l.groupby('Cluster3').apply(lambda x: x.loc[x['count'].idxmax()]).reset_index(drop=True)
df_l_plan = df_l_plan.apply(lambda x: x.sort_values().reset_index(drop=True))
df_l_plan

,L,count,Cluster3
0,5.0,1377.0,0.0
1,11.0,2680.0,1.0
2,29.0,3932.0,2.0
3,37.0,4291.0,3.0
4,42.0,5188.0,4.0
5,57.0,5527.0,5.0
6,66.0,5825.0,6.0
7,67.0,10816.0,7.0
8,74.0,11032.0,8.0
9,82.0,13645.0,9.0


In [23]:
df_color_plan = df_learn_show.groupby('Cluster2').apply(lambda x: x.loc[x['final_v'].idxmax()]).reset_index(drop=True)
df_color_plan

,Cluster,L,C,H,R,G,B,count,Cluster2,final_v
0,156.0,68.0,32.0,171.0,96.0,181.0,156.0,4724.0,0.0,10265952.0
1,0.0,53.0,66.0,45.0,210.0,87.0,47.0,2098.0,1.0,7292472.0
2,234.0,37.0,19.0,353.0,115.0,76.0,92.0,2902.0,2.0,2022398.0
3,72.0,38.0,33.0,274.0,46.0,90.0,142.0,2826.0,3.0,3515688.0
4,123.0,64.0,48.0,126.0,127.0,167.0,82.0,2086.0,4.0,6390816.0


In [24]:
df_plan = pd.DataFrame(columns=['L','C','H','R','G','B','count','count2','cluster2','cluster3'])

converter2 = cspace_converter("CIELCh","sRGB255")
# 将LCH颜色添加到DataFrame

lch = converter(RGB)
for idx, data in df_color_plan.iterrows():
    for idx2, data2 in df_l_plan.iterrows():
        L = data2['L']
        C = data['C']
        H = data['H']
        RGB = converter2((L, C, H))
        R = RGB[0]
        G = RGB[1]
        B = RGB[2]
        count = data['count']
        count2 = data2['count']
        cluster2 = data['Cluster2']
        cluster3 = data2['Cluster3']
        df_plan.loc[len(df_plan)] = [L,C,H,R,G,B,count,count2,cluster2,cluster3]

df_plan = df_plan.applymap(lambda x: 0 if x < 0 else x)
df_plan


/tmp/ipykernel_176929/1174046150.py:22: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



,L,C,H,R,G,B,count,count2,cluster2,cluster3
0,5.0,32.0,171.0,0.000000,29.834272,4.630058,4724.0,1377.0,0.0,0.0
1,11.0,32.0,171.0,0.000000,39.034313,21.879366,4724.0,2680.0,0.0,1.0
2,29.0,32.0,171.0,0.000000,79.959779,59.809800,4724.0,3932.0,0.0,2.0
3,37.0,32.0,171.0,0.000000,99.597795,78.145144,4724.0,4291.0,0.0,3.0
4,42.0,32.0,171.0,18.652037,112.176810,89.995916,4724.0,5188.0,0.0,4.0
5,57.0,32.0,171.0,66.847026,151.207206,127.133722,4724.0,5527.0,0.0,5.0
6,66.0,32.0,171.0,91.719046,175.483138,150.433485,4724.0,5825.0,0.0,6.0
7,67.0,32.0,171.0,94.457055,178.217591,153.065374,4724.0,10816.0,0.0,7.0
8,74.0,32.0,171.0,113.615977,197.558078,171.716814,4724.0,11032.0,0.0,8.0
9,82.0,32.0,171.0,135.630911,220.072864,193.499326,4724.0,13645.0,0.0,9.0


In [25]:
df_color_plan

,Cluster,L,C,H,R,G,B,count,Cluster2,final_v
0,156.0,68.0,32.0,171.0,96.0,181.0,156.0,4724.0,0.0,10265952.0
1,0.0,53.0,66.0,45.0,210.0,87.0,47.0,2098.0,1.0,7292472.0
2,234.0,37.0,19.0,353.0,115.0,76.0,92.0,2902.0,2.0,2022398.0
3,72.0,38.0,33.0,274.0,46.0,90.0,142.0,2826.0,3.0,3515688.0
4,123.0,64.0,48.0,126.0,127.0,167.0,82.0,2086.0,4.0,6390816.0


In [28]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=df_plan['cluster3'],
    y=df_plan['cluster2'],
    mode='markers',
    marker=dict(
        size=32,
        color=df_plan.apply(lambda row: f'rgb({row["R"]},{row["G"]},{row["B"]})', axis=1),
        opacity=0.8
    ),
    text=df_plan.apply(lambda row: f'R: {row["R"]}, G: {row["G"]}, B: {row["B"]}', axis=1)
))

fig.update_layout(
    title='配色方案',
    xaxis=dict(title='亮度'),
    yaxis=dict(title='颜色'),
    showlegend=False
)

fig.show()

In [27]:
import plotly.graph_objects as go

# Group the DataFrame by 'Cluster2' and select the row with the maximum 'L' value
df_max_l = df_learn_show.groupby('Cluster2').apply(lambda x: x.loc[x['final_v'].idxmax()]).reset_index(drop=True)

# Create the pie chart
fig = go.Figure(data=go.Pie(
    labels=df_max_l['Cluster2'],
    values=df_max_l['count'],
    marker=dict(
        colors=['rgb({},{},{})'.format(r, g, b) for r, g, b in zip(df_max_l['R'], df_max_l['G'], df_max_l['B'])]
    ),
    textinfo='none'
))

fig.update_layout(title='取色结果（如果图片颜色太少区别会很小，这不是bug是feature😡）', showlegend=False)
fig.show()
